In [86]:
import os
import numpy as np
import pandas as pd

In [87]:
#open the directroy and get the files from the directory in ../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/watch/accel/

In [88]:
def get_files_from_dir(directory) -> list:
    files = []
    for file in os.listdir(directory):
        print(file, "here we are")
        if file.endswith(".txt"):
            files.append(os.path.join(directory,file))
    return files

In [89]:
path = "../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/"
files = get_files_from_dir(path)
files

data_1631_accel_phone.txt here we are
data_1615_accel_phone.txt here we are
data_1628_accel_phone.txt here we are
data_1627_accel_phone.txt here we are
data_1603_accel_phone.txt here we are
data_1644_accel_phone.txt here we are
data_1643_accel_phone.txt here we are
data_1604_accel_phone.txt here we are
data_1639_accel_phone.txt here we are
.DS_Store here we are
data_1620_accel_phone.txt here we are
data_1612_accel_phone.txt here we are
data_1636_accel_phone.txt here we are
data_1605_accel_phone.txt here we are
data_1638_accel_phone.txt here we are
data_1642_accel_phone.txt here we are
data_1621_accel_phone.txt here we are
data_1613_accel_phone.txt here we are
data_1637_accel_phone.txt here we are
data_1630_accel_phone.txt here we are
data_1614_accel_phone.txt here we are
data_1629_accel_phone.txt here we are
data_1626_accel_phone.txt here we are
data_1645_accel_phone.txt here we are
data_1602_accel_phone.txt here we are
data_1623_accel_phone.txt here we are
data_1607_accel_phone.txt he

['../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1631_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1615_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1628_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1627_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1603_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1644_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1643_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1604_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_1639_accel_phone.txt',
 '../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone/accel/data_162

In [90]:
val = []
keys = []
# disallowed = ['C', 'F', 'G','H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S'] 
transform = {'A': 'wlk', 'B': 'jog', 'D': 'sit', 'E': 'std' } 
with open("../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/activity_key.txt", "r") as f:
    lines = f.read().splitlines()[:-1]
    # print(lines)
    for line in lines:
        
        split = line.split(" ", 1)
        end = line[line.index('= ')+2:]
        print(end)
        # if end in disallowed:
        #     print("skipped", end)
        #     continue
        keys.append(split[0])
        val.append(end)
    # keys = [line.split(" ", 1)[0] for line in lines] 
    # val = [line.split("= ",1)[1] for line in lines]
print(keys)
print(val)
# print(keys)
    

A
B
C
D
E
F
G
H
I
J
K
L
M
O
P
Q
R
S
['walking', 'jogging', 'stairs', 'sitting', 'standing', 'typing', 'teeth', 'soup', 'chips', 'pasta', 'drinking', 'sandwich', 'kicking', 'catch', 'dribbling', 'writing', 'clapping', 'folding']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'O', 'P', 'Q', 'R', 'S']


## Creating a dict where each file relates to a participant

In [91]:
def user_activity_split(file,should_print ) -> tuple:
    # each activity inside will be in the format (ndarray[[x,y,z], [x,y,z], ...], activity: ndarray[dws,dws,dws,...])
    #read he  text file into a dataaframe '
    opened = pd.read_csv(file, header=None, names=["user_id", "activity", "timestamp", "x", "y", "z"], sep=",")
    # remove the ";" at the end of the z column
    opened["z"] = opened["z"].str.replace(";","")
    res = []
    # get all the sensor data for an activity
    # the issue is the different activities so you need to have all the same activities as the other classes
    for activity in np.unique(opened["activity"]):
        # if activity in disallowed:
        #     continue
         
        # if activity in disallowed:
        #     print("skipped", activity)
        #     continue
       # add the sequences and labels to the list and make sure the x y z are of the float type
        transformed_activity = transform.get(activity, activity)
        seq = (opened[opened["activity"] == activity][["x", "y", "z"]].values.astype(np.float32))
        lab = np.array([transformed_activity] * len(seq))
        # print(lab)
        res.append((seq,lab)) 
    # print(sequences, labels)
    return res, np.unique(opened["activity"])

In [92]:
def process_user_data(files: list) -> dict:
    user_data = {}
    short_key = []
    for i, file in enumerate(files):
        
        user_tup, symbol = user_activity_split(file, i == 1)
        if i==0:
            mini_label= symbol
        
        user_data[i+1] = user_tup
    return user_data
res = process_user_data(files)


In [93]:
res

{1: [(array([[-3.2316895 ,  0.9601288 ,  1.223938  ],
          [-3.6880646 ,  2.4863586 ,  2.971283  ],
          [-2.923523  ,  8.615723  ,  5.365753  ],
          ...,
          [-0.11643982,  4.6147156 , -4.5350037 ],
          [-1.0953217 ,  6.4645844 , -5.5812073 ],
          [ 1.9113159 ,  9.717773  ,  1.6171265 ]], dtype=float32),
   array(['wlk', 'wlk', 'wlk', ..., 'wlk', 'wlk', 'wlk'], dtype='<U3')),
  (array([[  3.0844574 ,  -1.6601257 ,   3.1579437 ],
          [ -5.0481873 ,  -0.37516785, -15.005188  ],
          [  5.5169525 ,  15.839523  ,  -0.1363678 ],
          ...,
          [  7.151703  ,  14.151154  ,   8.735031  ],
          [  4.9741974 ,   3.6897736 ,   8.424454  ],
          [ -0.7993927 ,   1.7101135 ,  -2.040512  ]], dtype=float32),
   array(['jog', 'jog', 'jog', ..., 'jog', 'jog', 'jog'], dtype='<U3')),
  (array([[-2.3443604 ,  4.4835815 ,  0.7159271 ],
          [-1.2780914 ,  6.711334  ,  0.50427246],
          [ 2.4845886 , 13.92749   ,  3.8928223 ],
    

In [94]:
total = {}
total['label_list'] = val 
total['label_list_full_name'] = keys
total['has_null_class'] = False
total['user_split'] = res
total

{'label_list': ['A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'J',
  'K',
  'L',
  'M',
  'O',
  'P',
  'Q',
  'R',
  'S'],
 'label_list_full_name': ['walking',
  'jogging',
  'stairs',
  'sitting',
  'standing',
  'typing',
  'teeth',
  'soup',
  'chips',
  'pasta',
  'drinking',
  'sandwich',
  'kicking',
  'catch',
  'dribbling',
  'writing',
  'clapping',
  'folding'],
 'has_null_class': False,
 'user_split': {1: [(array([[-3.2316895 ,  0.9601288 ,  1.223938  ],
           [-3.6880646 ,  2.4863586 ,  2.971283  ],
           [-2.923523  ,  8.615723  ,  5.365753  ],
           ...,
           [-0.11643982,  4.6147156 , -4.5350037 ],
           [-1.0953217 ,  6.4645844 , -5.5812073 ],
           [ 1.9113159 ,  9.717773  ,  1.6171265 ]], dtype=float32),
    array(['wlk', 'wlk', 'wlk', ..., 'wlk', 'wlk', 'wlk'], dtype='<U3')),
   (array([[  3.0844574 ,  -1.6601257 ,   3.1579437 ],
           [ -5.0481873 ,  -0.37516785, -15.005188  ],
           [  5.5169525 ,  15.83952

## pickle the model to be read now

In [96]:
import pickle

In [97]:
#pickle the dictionary to the processed datasets folder
try:
    with open("../test_run/processed_datasets/wisdm_processed.pkl", "wb") as f:
        pickle.dump(total, f)
        print('saved') 
except Exception as e:
    print(e)

saved


## Automate the whole process


In [ ]:
all_modalities = {}

In [ ]:
# get files from a directory holding multiple folders
def get_modalities(source_dir) -> list:
    res = []
    for folder in os.listdir(source_dir):
        if folder.startswith("."):
            continue
        res.append(os.path.join(source_dir, folder))
    # print("done", res)
    return res


In [ ]:
def run_all(source_dir: str, modality_dict: dict) -> dict:
    modality = get_modalities(source_dir)
    # print(modality, "starting")
    for mod in modality:
        # each sensor source will have its own modalities
        files = get_files_from_dir(mod)
        # print(files, "files", mod)
        res = process_user_data(files)
        print(res, "res")
        modality_dict[mod[mod.rfind("/")+1:]] = res
    return modality_dict
    

In [ ]:
final = run_all("../test_run/original_datasets/wisdm/wisdm/wisdm-dataset/raw/phone", all_modalities)
print(final.keys(), "testing")